In [ ]:
import sys
sys.path.append('../Functions/py_functions/') # This path is so that within each function file, you can import the other function files with relative paths
sys.path.append('../') # This path is so that we can import the functions folder from the root directory compared to where this file is
from Functions.py_functions.constants import *
from Functions.py_functions.multi_sim import MultiSim
from Functions.py_functions.car_configuration import Car
from Functions.py_functions.las_solvers.octahedral import Octahedral_LAS
from Functions.py_functions.las_solvers.multi_layer import Multi_Layer_LAS
from Functions.py_functions.steady_state_solver.ls_optimize import LS_Solver
from Functions.py_functions.steady_state_solver.iterative import Iterative_Solver
from Functions.py_functions.gps_importer import *

In [ ]:
track_one = get_Lincoln_2017_AX_track_mixed(50)
track_two = get_MIS_2017_AX_1_track(50)
track_three = get_MIS_2017_AX_3_track(50)
tracks = [track_one, track_two, track_three]
# tracks = []
v_average = 15
mu_corr = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.001
# solver = LS_Solver()
solver = Iterative_Solver()
# las = Octahedral_LAS(solver=solver)
las = Multi_Layer_LAS(solver=solver)

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_lltd(rsd, k_c):
    total_roll_stiffness = 900 * FTLB_TO_NM
    car = Car()
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.k_c = k_c * FTLB_TO_NM
    car.description = f"rsd:{rsd:.2f} k_c:{k_c:.2f}"
    return car
# do a sweep of roll stiffness distribution and chassis roll stiffness
rsd_range = np.linspace(0.25, 0.5, 5)
k_c_r = np.linspace(500, 2500, 5) # np.array([1000]) #
sim = MultiSim(tracks, gen_car_lltd, rsd_range, k_c_r, "Roll Stiffness Distribution", "Chassis Roll Stiffness (lbf/deg)")

In [ ]:
def gen_car_cg(rsd, cg):
    total_roll_stiffness = 600 * FTLB_TO_NM
    car = Car(front_axle_weight=cg)
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.description = f"rsd:{rsd:.2f} cg placement:{cg:.2f}"
    return car
rsd_range = np.linspace(0.2, 0.4, 12)
cg_r = np.linspace(0.4, 0.6, 12)
sim = MultiSim(tracks, gen_car_cg, rsd_range, cg_r, "Roll Stiffness Distribution", "cg placement percent front")

In [ ]:
def gen_car_mass(rsd, mass):
    total_roll_stiffness = 600 * FTLB_TO_NM
    car = Car(mass=mass)
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.description = f"rsd:{rsd:.2f} mass:{mass:.2f}"
    car.update_car()
    return car
rsd_range = np.linspace(0.35, 0.6, 12)
mass_r = np.linspace(200, 300, 5)
sim = MultiSim(tracks, gen_car_mass, rsd_range, mass_r, "Roll Stiffness Distribution", "Mass (kg)")

In [ ]:
def gen_car_toe(f_toe, r_toe):
    car = Car()
    car.toe_front = f_toe
    car.toe_rear = r_toe
    car.description = f"f_toe:{car.toe_front:.2f} r_toe:{car.toe_rear:.2f}"
    return car
f_toe_r = np.linspace(-10, 5, 20)
r_toe_r = np.linspace(-6, 6, 20)
sim = MultiSim(tracks, gen_car_toe, f_toe_r, r_toe_r, "Front Toe (deg)", "Rear Toe (deg)")

In [ ]:
# here we make a function that generates a car with a given roll stiffness distribution and chassis roll stiffness
# the first and second arguments take the values of the first and second sweep variables that you give to run_sim
def gen_car_rtw(rsd, rtw):
    total_roll_stiffness = 900 * FTLB_TO_NM
    car = Car()
    car.k_f = total_roll_stiffness * rsd
    car.k_r = total_roll_stiffness * (1-rsd)
    car.rear_track = rtw * IN_TO_M
    car.description = f"rsd:{rsd:.2f} k_c:{rtw:.2f}"
    return car
# do a sweep of roll stiffness distribution and chassis roll stiffness
rsd_range = np.linspace(0.25, 0.5, 10)
rtw_r = np.linspace(46, 50, 5) # np.array([1000]) #
sim = MultiSim(tracks, gen_car_rtw, rsd_range, rtw_r, "Roll Stiffness Distribution", "Rear Track Width (in)")

In [ ]:

sim.run_sim(las=las, convergence_target=target, mu=mu_corr, sim_type='qts')
sim.plot_tracks()
sim.plot_LAS_corners()
sim.plot_LLTD()

In [ ]:
sim.run_skidpad(sr_lim=0.2)
sim.plot_skidpad()